# Check what you are voting for with PyTezos

In [1]:
import sys
assert sys.version_info.major == 3 and sys.version_info.minor >= 6

In [2]:
import os
sys.path.insert(1, os.path.abspath('..'))

## Current voting phase

Say, you don't read any news and all you have is public node access and documentation.  
How to determine what is the current voting phase?  

In [3]:
from pytezos.rpc import mainnet

In [4]:
mainnet.head.votes.current_period_kind()

'proposal'

Great, we are at the first stage, now we want to know when did it start and when it will ends.  
We can get this information from the block metadata:

In [39]:
level_info = mainnet.head.metadata.get('level')
level_info

{'level': 349140,
 'level_position': 349139,
 'cycle': 85,
 'cycle_position': 979,
 'voting_period': 10,
 'voting_period_position': 21459,
 'expected_commitment': False}

Get precise boundaries in levels and rough estimation in days

In [40]:
import pendulum

In [93]:
start_level = level_info['level'] - level_info['voting_period_position']
start_level

327681

In [98]:
end_level = start + 8 * 4096 - 1  # eight cycles of 4096 blocks
end_level

360448

In [118]:
start_dt = pendulum.parse(mainnet.blocks[start_level].header.get('timestamp'))
time_past = (pendulum.now() - start_dt)
print(round(time_past.total_days(), 1), 'days passed')

15.5 days passed


In [119]:
time_left = (end_level - level_info['level']) / (level_info['level'] - start_level) * time_past
print(round(time_left.total_days(), 1), 'days left')

8.2 days left


## Current proposals

In [99]:
proposals = mainnet.head.votes.proposals()
proposals

[['Psd1ynUBhMZAeajwcZJAeq5NrxorM6UCU4GJqxZ7Bx2e9vUWB6z', 5176],
 ['Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd', 11770]]

Let's examine one of proposals

In [100]:
proposal_id = proposals[1][0]
proposal_id

'Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd'

### Injection operation

It's interesting to find the author of the proposal.  
In order to do that we have to search for the first `proposals` operation for this particular proposal_id.

In [120]:
from pytezos.rpc.search import SearchChain

In [121]:
sc = SearchChain.from_chain(mainnet.main)

Thanks to the statefullness of the Tezos blockchain we can perform a binary search inside the voting period.  
The algorithm searches for the first level where number of votes changed from 0 to non-zero.

In [122]:
operation = sc.find_proposal_inject_operation(proposal_id)

2019-03-12 15:03:47.543 | DEBUG    | pytezos.rpc.search:bisect:29 - 714 at level 338427
2019-03-12 15:03:48.414 | DEBUG    | pytezos.rpc.search:bisect:29 - 23 at level 333054
2019-03-12 15:03:49.377 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 330367
2019-03-12 15:03:50.143 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 331710
2019-03-12 15:03:50.950 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332382
2019-03-12 15:03:51.784 | DEBUG    | pytezos.rpc.search:bisect:29 - 11 at level 332718
2019-03-12 15:03:52.667 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332550
2019-03-12 15:03:54.077 | DEBUG    | pytezos.rpc.search:bisect:29 - 11 at level 332634
2019-03-12 15:03:54.934 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332592
2019-03-12 15:03:55.721 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332613
2019-03-12 15:03:56.601 | DEBUG    | pytezos.rpc.search:bisect:29 - 0 at level 332623
2019-03-12 15:03:57.418 | DEBUG    | pytezos.rpc.

In [123]:
operation()

{'protocol': 'PsddFKi32cMJ2qPjf43Qv5GDWLDPZb3T3bF6fLKiF5HtvHNU7aP',
 'chain_id': 'NetXdQprcVkpaWU',
 'hash': 'onydFJLWdGhfKNBfbnSLmqDu93j9NRimkbQm9WqLWYG8eyZUyTF',
 'branch': 'BL53WJx6xPn6rnTnWZmpNaWGAQqU8HTwRTDqVDmthYsxUTBewo9',
 'contents': [{'kind': 'proposals',
   'source': 'tz1fNdh4YftsUasbB1BWBpqDmr4sFZaPNZVL',
   'period': 10,
   'proposals': ['Pt24m4xiPbLDhVgVfABUjirbmda3yohdN82Sp9FeuAXJ4eV9otd',
    'Psd1ynUBhMZAeajwcZJAeq5NrxorM6UCU4GJqxZ7Bx2e9vUWB6z'],
   'metadata': {}}],
 'signature': 'sigvUqvh7rBS8yAoE5RMieQaD5hvg9NsLeJ4kTnXdK1tXXyrHL8mX7E3KCm9q9YgYbJn3edhcUiZjdU3xNhVPEUPkSGVNbi9'}

We can perform some checks on this operation, such as signature validation:

In [124]:
operation.verify_signature()

The only thing we can learn about the submitter's identity is his public key:

In [126]:
mainnet.get_public_key(operation.source())

'edpktyybTTrLm2rk6bn7xtgY9t2Kgt9GnqZxEcSrunm4vKWTF9ES9o'

In [141]:
timestamp = mainnet.blocks[operation.get('branch')].header.get('timestamp')
inject_dt = pendulum.parse(timestamp)
print('Ins' (inject_dt - start_dt).days + 1)

4

In [133]:
print(round((pendulum.now() - inject_dt).total_days(), 1), 'days passed')

12.0 days passed


### Current protocol source

In [ ]:
current_proto = mainnet.protocols[operation.get('protocol')]
current_proto

Load files from blockchain, encode (without protocol-compiler binary) and get hash

In [ ]:
current_hash = current_proto.calculate_hash()
current_hash

In [ ]:
assert current_hash == operation.get('protocol')

### Source code

Unfortunately it's practically impossible to get proposal source from the blockchain.
We could get lucky and find the submitter's node (in case it's public) which has to know this proto (according to the docs).
The other option is try to find a node in zeronet which has participated in the voting rehearsal.

In [ ]:
tar_url = 'https://blog.nomadic-labs.com/files/Athens_proposal_A.tar'

In [ ]:
from pytezos.rpc.protocol import Protocol

Loading sources and convert them to the internal format (as in blockchain)

In [ ]:
athens_a = Protocol.from_uri(tar_url)

In [ ]:
proto_id = athens_a.calculate_hash()
proto_id

In [ ]:
assert proposal_id == proto_id

### Get unified diff

In [ ]:
patch = current_proto.diff(athens_a, context_size=3)

Generate github-like side-by-side patch views, powered by diff2html.js

In [ ]:
patch.export_html(output_path='report.html')

[View report](./report.html)

### Get all voting operations for this proposal

In [ ]:
for operation in sc.find_proposal_votes_operations(proposals[0][0]):
    print(operation())
    break  # this can take a while

Search algorithm works as follows:
1. Split block interval into equal chunks
2. Determine which of the intervals contain state changes
3. For each interval run binary search
4. If there are several changes inside single interval, run binary search again

It's obvious that the search space can be easily splitted and processed independently, i.e parallelized. 

### Storing protocol diff

In [ ]:
ctxless_patch = current_proto.diff(athens_a, context_size=0)

In [ ]:
ctxless_patch.export_tar('diff.tar.bz2')

In [ ]:
os.path.getsize('diff.tar.bz2')

As we pointed earlier there is no convenient way to get proposal source from the blockchain. This can be implemented via smart-contract. But it's more reasonable to store compressed code diff rather than full source.

Compare 13 KB vs 1.2 MB (original tar)

### Applying protocol diff

In [ ]:
proto = current_proto.apply(ctxless_patch)

In [ ]:
assert proto_id == proto.calculate_hash()